# Práctico 5 informe

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
np.set_printoptions(suppress=True,precision=3)
from matplotlib.patches import FancyArrowPatch

In [7]:
# Importamos los datos utilizando pandas
data=pd.read_csv("./data.csv")

# Creo la tabla de candidatos a mano
candidatos=pd.DataFrame(
    [
        [1,'Oscar Andrade', 'Frente Amplio'],
        [2,'Mario Bergara', 'Frente Amplio'],
        [3,'Carolina Cosse', 'Frente Amplio'],
        [4,'Daniel Martínez', 'Frente Amplio'],
        [5,'Verónica Alonso', 'Partido Nacional'],
        [6,'Enrique Antía', 'Partido Nacional'],
        [8,'Carlos Iafigliola', 'Partido Nacional'],
        [9,'Luis Lacalle Pou', 'Partido Nacional'],
        [10,'Jorge Larrañaga', 'Partido Nacional'],
        [11,'Juan Sartori', 'Partido Nacional'],
        [12,'José Amorín', 'Partido Colorado'],
        [13,'Pedro Etchegaray', 'Partido Colorado'],
        [14,'Edgardo Martínez', 'Partido Colorado'],
        [15,'Héctor Rovira', 'Partido Colorado'],
        [16,'Julio María Sanguinetti', 'Partido Colorado'],
        [17,'Ernesto Talvi', 'Partido Colorado'],
        [18,'Pablo Mieres', 'La Alternativa'],
        [19,'Gonzalo Abella', 'Unidad Popular'],        
        [20,'Edgardo Novick', 'Partido de la Gente'],
        [21,'Cèsar Vega', 'PERI'],
        [22,'Rafael Fernández', 'Partido de los Trabajadores'],
        [23,'Justin Graside', 'Partido Digital'],        
        [24,'Gustavo Salle', 'Partido Verde'],
        [25,'Carlos Techera', 'Partido de Todos']
    ],
    columns=['candidatoId','name','party'],
)

data=data.merge(candidatos,on=['candidatoId'])

# Sólo por si necesita, cargamos un diccionario con el texto de cada pregunta
preguntas={
    '1': 'Controlar la inflación es más importante que controlar el desempleo. ',
    '2': 'Hay que reducir la cantidad de funcionarios pùblicos',
    '3': 'Deberia aumentar la carga de impuestos para los ricos.',
    '4': 'El gobierno no debe proteger la industria nacional, si las fábricas no son competitivas esta bien que desaparezcan.',
    '5': 'La ley de inclusión financiera es positiva para la sociedad. ',
    '6': 'Algunos sindicatos tienen demasiado poder. ',
    '7': 'Cuanto más libre es el mercado, más libre es la gente. ',
    '8': 'El campo es y debe ser el motor productivo de Uruguay. ',
    '9': 'La inversión extranjera es vital para que Uruguay alcance el desarrollo. ',
    '10': 'Los supermercados abusan del pueblo con sus precios excesivos. ',
    '11': 'Con la vigilancia gubernamental (escuchas telefonicas, e-mails y camaras de seguridad) el que no tiene nada que esconder, no tiene de que preocuparse. ',
    '12': 'La pena de muerte debería ser una opción para los crímenes mas serios. ',
    '13': 'Uruguay debería aprobar más leyes anti corrupción y ser más duro con los culpables. ',
    '14': 'Las FF.AA. deberían tener un rol activo en la seguridad pública. ',
    '15': 'Las carceles deberían ser administradas por organizaciones privadas. ',
    '16': 'Hay que aumentar el salario de los policias significativamente. ',
    '17': 'Para los delitos más graves hay que bajar la edad de imputabilidad a 16 años. ',
    '18': 'Uruguay no necesita un ejército. ',
    '19': 'Uruguay es demasiado generoso con los inmigrantes. ',
    '20': 'La ley trans fue un error. ',
    '21': 'El feminismo moderno no busca la igualdad sino el poder. ',
    '22': 'Para la ley no deberia diferenciarse homicidio de femicidio. ',
    '23': 'La separación de estado y religión me parece importante. ',
    '24': 'La legalización de la marihuana fue un error. ',
    '25': 'La legalización del aborto fue un error. ',
    '26': 'El foco del próximo gobierno debe ser mejorar la educación pública. '
}

# Ordeno los datos por partido y luego por candidato

data = data.sort_values(by=['party','name'])
# Para PCA solo usamos las preguntas
questions = data.iloc[:,2:28]
print(questions)

       1  2  3  4  5  6  7  8  9  10  ...  17  18  19  20  21  22  23  24  25  \
19998  4  4  3  4  5  5  3  2  3   4  ...   1   4   1   1   1   1   5   1   1   
19999  3  4  5  2  4  4  2  2  4   4  ...   1   5   2   1   1   2   5   1   1   
20000  2  3  4  2  4  4  2  2  4   3  ...   1   4   2   1   2   2   5   1   1   
20001  4  4  3  4  4  4  3  3  4   4  ...   4   4   2   3   3   2   5   2   2   
20002  4  5  3  3  4  5  5  4  3   3  ...   4   2   3   2   4   4   5   2   1   
20003  2  5  5  1  4  5  1  1  5   5  ...   1   4   2   1   1   1   5   1   1   
20004  3  5  5  1  2  5  4  4  5   3  ...   4   5   2   2   4   5   5   1   1   
20005  2  2  5  2  3  2  1  4  4   4  ...   2   4   1   1   1   1   5   1   1   
20006  3  2  5  3  4  3  2  3  4   4  ...   2   3   2   1   2   5   5   1   1   
20007  3  4  5  2  4  4  1  2  4   2  ...   1   5   1   1   1   1   5   1   1   
20008  1  2  5  1  5  1  1  2  4   5  ...   1   4   1   1   2   4   5   1   1   
20009  4  4  5  3  4  4  4  

## KMeans

In [18]:
from src.kmeans import KMeans
from time import time
import datetime
from sklearn.metrics import adjusted_rand_score

In [24]:
start_time = time()
kmeans = KMeans(questions, cluster_count = 11)
clusters, centroids, data_cluster_idx = kmeans.train()
elapsed_time = time() - start_time

print(f'TOTAL TIME: {datetime.timedelta(seconds=elapsed_time)}')
#print(clusters)
print(centroids)
print(data_cluster_idx)

##### COST FUNCTION #####
3238910.3778410656
##### RUNS #####
10
TOTAL TIME: 0:00:50.502921
[array([2.693, 3.266, 2.948, 2.529, 2.787, 3.579, 2.67 , 3.242, 3.219,
       3.11 , 2.574, 2.163, 3.676, 2.781, 2.712, 3.23 , 2.982, 2.773,
       2.89 , 3.732, 3.709, 3.613, 3.307, 3.898, 4.055, 3.89 ]), array([3.23 , 4.408, 2.833, 2.755, 3.806, 4.453, 3.188, 3.2  , 4.04 ,
       3.232, 2.463, 2.104, 4.496, 2.195, 3.053, 3.625, 2.979, 2.686,
       2.019, 1.995, 2.276, 3.222, 4.832, 1.881, 1.272, 4.555]), array([3.279, 4.568, 2.234, 2.763, 2.284, 4.774, 4.109, 4.117, 4.281,
       3.201, 3.205, 3.197, 4.684, 3.798, 3.298, 3.869, 4.503, 1.993,
       3.122, 4.129, 4.4  , 4.475, 4.206, 4.091, 3.32 , 4.523]), array([3.078, 3.007, 4.2  , 2.077, 4.345, 3.877, 2.673, 2.892, 3.871,
       3.915, 3.58 , 1.853, 4.311, 2.028, 2.885, 3.988, 2.308, 3.354,
       1.915, 1.3  , 2.194, 2.873, 4.798, 1.487, 1.337, 4.487]), array([3.413, 4.567, 2.166, 2.822, 2.367, 4.742, 4.087, 4.307, 4.328,
       3.832, 3.6

### Adjusted Random Index (ARI)

In [25]:
party_map = {
        'Frente Amplio': 0,
        'Partido Nacional': 1,
        'Partido Colorado': 2,
        'La Alternativa': 3,
        'Unidad Popular': 4,
        'Partido de la Gente': 5,
        'PERI': 6,
        'Partido de los Trabajadores': 7,
        'Partido Digital': 8,
        'Partido Verde': 9,
        'Partido de Todos': 10
}

numeric_parties  = data.party.map(party_map)
ari = adjusted_rand_score(numeric_parties,data_cluster_idx)
print(ari)

0.14729454236766568


## PCA - Principal Component Analysis

Explicar que es y para que sirve

Explicar dataset 

Para comenzar, calcularemos media y varianza de cada una de las columnas. 

La media es, por supuesto:

$$ \bar{X} = \frac{\sum_{i=n}^n X_i }{n} $$

Y la varianza mide qué tanto los datos se alejan de la media: 

$$ var(X) = \frac{\sum_{i=n}^n (X_i - \bar{X})(X_i - \bar{X}) }{n-1} $$



In [ ]:
mean=np.mean(data,axis=1) 
std=np.std(data,axis=1)
var=np.var(data,axis=1)

print ("Media:",mean)
print ("Varianza:",var)
print ("Desviación estándar:", std)





La desviación estándar es simplemente la raíz cuadrada de la varianza (la utlizamos porque es comparable con la media).  Observemos que al tomar cuadrados, las desviaciones positivas y negativas de la media _no_ se cancelan. 

Restamos la media a cada dimensión para que nuestro conjunto quede centrado en en el origen

In [ ]:
questions_adjust = questions - questions.mean(axis=0)
print (questions_adjust)

mean=np.mean(questions_adjust,axis=0) 
std=np.std(questions_adjust,axis=0)
var=np.var(questions_adjust,axis=0)

print ("Media:",mean)
print ("Varianza:",var)
print ("Desviación estándar:", std)

In [ ]:


'''
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10   
ax.plot(X_r[0,0:int(num_instances/2)] X_r[1,0:int(num_instances/2)], X_r[2,0:int(num_instances/2)], 'o', markersize=8, color='red', alpha=0.5, label='class1')
ax.plot(X_r[0,int(num_instances/2):num_instances], X_r[1,int(num_instances/2):num_instances], X_r[2,int(num_instances/2):num_instances], 'o', markersize=8, color='blue', alpha=0.5, label='class2')

plt.show()
'''


Volvemos a calcular media y varianza:

La covarianza se define como:

$$ cov(X, Y) = = \frac{\sum_{i=n}^n (X_i - \bar{X})(Y_i - \bar{Y}) }{n-1} $$


y es una medida estadística que mide cómo se relacionan dos dimensiones. Si la covarianza es positiva, indica que ambas dimensiones crecen juntas, mientras que si es negativa, indica que cuando una crece, la otra decrece. Si la covarianza es 0, entonces las dimensiones son independientes (es decir, modificar una no afecta a la otra). 

La matriz de covarianza tiene en cada elemento de la diagonal la varianza de la dimensión correspondiente, y en su elemento $ c_{ij}$ la covarianza de las dimensiones $i$ y $j$ (nótese que $cov(i,j)=cov(j,i)$ y por lo tanto la matriz es simétrica

In [ ]:
cvm=np.cov(questions_adjust, rowvar = False)
print (cvm)

Como la matriz es simétrica, sabemos que su valores propios son reales, así que vamos a calcularlos, y de esta forma obtendremos las direcciones en las que hay mayor varianza. Utilizamos para eso la función eig de Numpy.

In [ ]:
eig_val_cov, eig_vec_cov = np.linalg.eig(cvm)

for i in range(len(eig_val_cov)):
    eigvec_cov = eig_vec_cov[:,i].reshape(1,26).T

    print('Valor propio {} de la matriz de covarianza {}'.format(i+1, eig_val_cov[i]))
    print('Vector propio:')
    print(eigvec_cov)


Los vectores propios obtenidos nos indican las direcciones en las que los datos varían más. El vector con el mayor valor propio es el _componente principal_ de la matriz, es decir que muestra la dirección en la que los datos están más dispersos. Y así podemos continuar, viendo las dimensiones donde hay más variación, ordenando los vectores propios según sus valores propios. 

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_val_cov[i]), eig_vec_cov[:,i]) for i in range(len(eig_val_cov))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
for i in eig_pairs:
    print(i)
    #    print(i[0])



Reduciremos las dimensiones a dos para poder visualizarlo graficamente. Notar que hay un gran salto entre el valor propio asociado al primer componente pricipal y el segundo: 19 vs 3.37. Los que nos puede hacer intuir que es este el realmente determinante.



In [ ]:
# Nos quedamos con los primeros componentes principales
matrix_w = np.array([eig_pairs[0][1], eig_pairs[1][1]]).T
print('Matrix W:\n', matrix_w)
print('Questions\n', questions_adjust.T)


Pasamos a la base de vectores propios

In [ ]:
# Transform instance to the new subspace

transformed = (matrix_w.T @ questions_adjust.T).T
#  We will need the parties and candidates to plot
transformed = pd.concat([transformed, data[['party', 'name']]], axis=1)
print (transformed)

Hemos obtenido un nuevo dataset con una dimensión menos, pero que respeta (tanto como es posible) la dispersión de los datos, porque tiene las dos dimensiones de mayor varianza.

Al ver los valores propios, observamos que uno de ellos era mucho mayor que los otros, así que puede tener sentido ver qué sucede si quitamos una dimensión más: 

In [ ]:
num_instances = len(transformed.index)
plt.plot(transformed[transformed['party'] == 'Frente Amplio'].iloc[:,0],transformed[transformed['party'] == 'Frente Amplio'].iloc[:,1], 'o', markersize=7, color='yellow', alpha=1, label='Frente Amplio')
plt.plot(transformed[transformed['party'] == 'Partido Nacional'].iloc[:,0],transformed[transformed['party'] == 'Partido Nacional'].iloc[:,1], 'o', markersize=7, color='skyblue', alpha=1, label='Partido Nacional')
plt.plot(transformed[transformed['party'] == 'Partido Colorado'].iloc[:,0],transformed[transformed['party'] == 'Partido Colorado'].iloc[:,1], 'o', markersize=7, color='red', alpha=1, label='Partido Colorado')
plt.plot(transformed[transformed['party'] == 'La Alternativa'].iloc[:,0],transformed[transformed['party'] == 'La Alternativa'].iloc[:,1], 'o', markersize=7, color='black', alpha=1, label='La Alternativa')
plt.plot(transformed[transformed['party'] == 'Unidad Popular'].iloc[:,0],transformed[transformed['party'] == 'Unidad Popular'].iloc[:,1], 'o', markersize=7, color='grey', alpha=1, label='Unidad Popular')
plt.plot(transformed[transformed['party'] == 'Partido de la Gente'].iloc[:,0],transformed[transformed['party'] == 'Partido de la Gente'].iloc[:,1], 'o', markersize=7, color='yellowgreen', alpha=1, label='Unidad Popular')
plt.plot(transformed[transformed['party'] == 'PERI'].iloc[:,0],transformed[transformed['party'] == 'PERI'].iloc[:,1], 'o', markersize=7, color='green', alpha=1, label='PERI')
plt.plot(transformed[transformed['party'] == 'Partido de los Trabajadores'].iloc[:,0],transformed[transformed['party'] == 'Partido de los Trabajadores'].iloc[:,1], 'o', markersize=7, color='purple', alpha=1, label='Partido de los Trabajadores')
plt.plot(transformed[transformed['party'] == 'Partido Digital'].iloc[:,0],transformed[transformed['party'] == 'Partido Digital'].iloc[:,1], 'o', markersize=7, color='orange', alpha=1, label='Partido Digital')
plt.plot(transformed[transformed['party'] == 'Partido Verde'].iloc[:,0],transformed[transformed['party'] == 'Partido Verde'].iloc[:,1], 'o', markersize=7, color='springgreen', alpha=1, label='Partido Verde')
plt.plot(transformed[transformed['party'] == 'Carlos Techera'].iloc[:,0],transformed[transformed['party'] == 'Carlos Techera'].iloc[:,1], 'o', markersize=7, color='cyan', alpha=1, label='Carlos Techera')
#plt.xlim([-4,4])
#plt.ylim([-4,4])
plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.title('Instancias transformadas (con etiquetas)')

plt.show()



In [ ]:
# Just one dimension 
matrix_w2 = np.hstack((eig_pairs[0][1].reshape(3,1),))
print('Matrix W:\n', matrix_w2)



In [ ]:
transformed2 = np.dot(X_r.T,matrix_w2).transpose()
print (transformed2)

In [ ]:
plt.plot(transformed2[0,0:int(num_instances/2)],np.zeros(int(num_instances/2)), 'o', markersize=7, color='blue', alpha=0.5, label='hombres')
plt.plot(transformed2[0,int(num_instances/2):num_instances],np.zeros(int(num_instances/2)), '^', markersize=7, color='red', alpha=0.5, label='mujeres')
#plt.xlim([-4,4])
#plt.ylim([-4,4])
plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.title('Instancias transformadas (con etiquetas)')

plt.show()